In [2]:
!pip install "mlflow<2.0.0"

  Using cached mlflow-1.30.1-py3-none-any.whl.metadata (11 kB)
  Using cached cloudpickle-2.2.1-py3-none-any.whl.metadata (6.9 kB)
  Using cached databricks_cli-0.18.0-py2.py3-none-any.whl.metadata (4.0 kB)
  Using cached entrypoints-0.4-py3-none-any.whl.metadata (2.6 kB)
  Using cached GitPython-3.1.44-py3-none-any.whl.metadata (13 kB)
  Using cached protobuf-4.25.5-cp37-abi3-macosx_10_9_universal2.whl.metadata (541 bytes)
  Using cached pytz-2022.7.1-py2.py3-none-any.whl.metadata (21 kB)
  Using cached packaging-21.3-py3-none-any.whl.metadata (15 kB)
  Using cached importlib_metadata-5.2.0-py3-none-any.whl.metadata (5.0 kB)
  Using cached sqlparse-0.5.3-py3-none-any.whl.metadata (3.9 kB)
  Using cached alembic-1.14.0-py3-none-any.whl.metadata (7.4 kB)
  Using cached docker-6.1.3-py3-none-any.whl.metadata (3.5 kB)
  Using cached flask-2.3.3-py3-none-any.whl.metadata (3.6 kB)
  Using cached pandas-1.5.3.tar.gz (5.2 MB)
  Installing build dependencies ... done
  Getting requirements to 

In [1]:
from minio import Minio

minio_client = Minio(
    "localhost:57100",
    access_key="mlops",
    secret_key="mlops123",
    secure=False,
)

if not minio_client.bucket_exists('mlflow'):
    minio_client.make_bucket('mlflow')

In [2]:
import mlflow

mlflow.set_tracking_uri("http://localhost:59000")

for i in range(3):
    with mlflow.start_run():
        mlflow.log_param("trial", i)
        mlflow.log_metric("metric", i+1)

In [3]:
from sklearn.ensemble import AdaBoostRegressor
from sklearn.datasets import make_regression

import numpy as mp
import pandas as pd

class MyModel:

    def __init__(self, n_features):
        self.n_features = n_features
        self.model = AdaBoostRegressor(random_state=0, n_estimators=100)

    def fit(self):
        x, y = make_regression(
            n_features=self.n_features,
            n_informative=2,
            random_state=0,
            shuffle=False
        )
        self.model.fit(x, y)

    def predict(self, input_sample):
        pd_input = pd.DataFrame(input_sample)
        pred = self.model.predict(pd_input)
        return pd.DataFrame(pred)

input_df = pd.DataFrame(data=[[0, 0.0]], columns=["co2", "temperature"])

# create and train model
my_model = MyModel(
    n_features=len(input_df.columns)
)

In [4]:
mlflow.sklearn.autolog()

with mlflow.start_run() as run:
    my_model.fit()

2025/01/07 21:21:44 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of sklearn. If you encounter errors during autologging, try upgrading / downgrading sklearn to a supported version, or try upgrading MLflow.


AttributeError: module 'sklearn.metrics' has no attribute 'SCORERS'